# ewx jr

In [1]:
import logging
import socket
import requests
import time
import json
import os
import sys
import re
import urllib
import flask
import httplib2
import pprint
import googleapiclient
import oauth2client
import energyworx_client
import energyworx_public
from dateutil.parser import parse
import google.oauth2.credentials

from retrying import retry
from datetime import datetime
from energyworx_client import client
from energyworx_public import domain, rule, base, enums
from googleapiclient import discovery
from google_auth_oauthlib import flow
from requests.auth import HTTPBasicAuth
from oauth2client import GOOGLE_TOKEN_URI
from googleapiclient.errors import HttpError
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
from oauth2client.client import GoogleCredentials, OAuth2Credentials

pp = pprint.PrettyPrinter(indent = 1)
print('imported modules.')

imported modules.


## run thru google oauth2 flow

In [2]:
def google_oauth_flow():

    #read secret file google api
    with open('F:\\client_secret_symbolic_bit.apps.googleusercontent.com.json', 'r') as f:
        secret_file = json.load(f)
        secret = secret_file['web']
    
    print('read in super-secret google api credentials.')
    print('')
    
    #run oauth consent
    SCOPES = ['https://www.googleapis.com/auth/userinfo.email', 'openid']
    oauth = OAuth2Session(secret['client_id'], redirect_uri=secret['redirect_uris'][0], scope=SCOPES)
    authorization_url, state = oauth.authorization_url('https://accounts.google.com/o/oauth2/auth', access_type="offline", prompt="consent")

    print('Please go to %s and authorize access.' % authorization_url)
    print('')
    
    authorization_response = input('Enter the full callback URL')
    print('')
    
    #get access token
    token = oauth.fetch_token('https://accounts.google.com/o/oauth2/token', authorization_response=authorization_response, client_secret=secret['client_secret'])
    print('fetched extra super-secret token.')
    
    token_exp = datetime.fromtimestamp(token['expires_at'])
    print('Bearer token expires at: {}.'.format(token_exp.strftime('%m/%d/%Y %H:%M:%S')))
    print('')
    
    #write modified client secret
    secret['type'] = 'authorized_user'
    secret['refresh_token'] = token['refresh_token']

    with open('F:\\super-secret-google-shit.json', 'w') as g:
        json.dump(secret, g)
    
    print('wrote new client secret to F:\\super-secret-google-shit.json')
    
    os.environ['JPY_USER'] = "True"
    os.environ['REFRESH_TOKEN'] = secret['refresh_token']
    os.environ['CLIENT_ID'] = secret['client_id']
    os.environ['CLIENT_SECRET'] = secret['client_secret']
    os.environ['TOKEN_URI'] = secret['token_uri']
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'F:\\super-secret-google-shit.json'
    print('set necessary os environment variables.')
    
    return(oauth, secret, token)

In [3]:
oauth, secret, token = google_oauth_flow()

read in super-secret google api credentials.

Please go to https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=348739208671-qm0broa5bkddp3t85qibse6l4kqe58fe.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fstevenhurwitt.com%2Fhub%2Foauth_callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid&state=qr1cPM8UHIpjMY3LzNN3OxapiXaExk&access_type=offline&prompt=consent and authorize access.

Enter the full callback URLhttps://stevenhurwitt.com/hub/oauth_callback?state=qr1cPM8UHIpjMY3LzNN3OxapiXaExk&code=4%2F2QGnS3YHui-omBnF2WZN6-megyaGPYd6aOMO9sF3q6lOXm7BubThjItsuRpwWyC9JN6_53Al6szh4FVi3DgZOto&scope=email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+openid&authuser=1&prompt=consent#

fetched extra super-secret token.
Bearer token expires at: 07/28/2020 18:54:45.

wrote new client secret to F:\super-secret-google-shit.json
set necessary os environment variables.


## test request

In [4]:
r = oauth.get('https://www.googleapis.com/oauth2/v1/userinfo')
r.json()

{'id': '116108006493438593342',
 'email': 'steven.hurwitt@external.engie.com',
 'verified_email': True,
 'picture': 'https://lh4.googleusercontent.com/-11HvE0chTJs/AAAAAAAAAAI/AAAAAAAAAAA/7qsrZ7S5SPw/photo.jpg'}

# EWX API credentials

In [5]:
namespace = 'na.engie.com'
test_namespace = 'na.engie_test.com'
default_api_root = 'https://ewx-live.appspot.com'
other = 'cloud.app.energyworx.com/_ah/api'

creds = GoogleCredentials.get_application_default()

#create ewx client
ewx = client.EWX(namespace, default_api_root, creds)

Using namespace = na.engie.com and api_root = https://ewx-live.appspot.com
Creating Externalservice client
Discovering services using https://ewx-live.appspot.com/_ah/api/discovery/v1/apis/ewx/v1/rest.
Client created


## get datasource

In [8]:
#ewx.get_datasource('NEPOOL_BECO_25917091008')

## try api request

In [6]:
X_API_KEY="626E2AA48CE14D5EA2B4DB8F2052B9FC"
X_BEARER_TOKEN='Bearer ' + token['access_token']

headers = {"Accept": "application/json", "x-namespace" : namespace}
auth = HTTPBasicAuth("apikey", X_API_KEY)
auth2 = HTTPBasicAuth('Authorization', str(X_BEARER_TOKEN))

response = requests.get('https://ewx-live.appspot.com/_ah/api/ewx/v1/datasource/get/NEPOOL_BECO_25917091008', auth = auth, headers = headers)
#response.json()

## load datasource json

In [2]:
with open('NEPOOL_MECO_0219222018_ingest.json', 'r') as h:
    ds_dict = json.load(h)

pp.pprint(ds_dict)

{'channels': [{'classifier': 'DELIVERY_IDR',
               'datapointType': 'interval',
               'description': 'Automatic generated channel',
               'id': 'DELIVERY_IDR',
               'isSource': True,
               'name': 'Delivery Idr',
               'unitType': 'kWh'},
              {'classifier': 'DELIVERY_SCALAR',
               'datapointType': 'gauge',
               'description': 'Automatic generated channel',
               'id': 'DELIVERY_SCALAR',
               'isSource': True,
               'name': 'Delivery Scalar',
               'unitType': 'kWh'},
              {'classifier': 'FORECAST',
               'datapointType': 'interval',
               'description': 'No description',
               'id': 'FORECAST',
               'isSource': False,
               'name': 'No name',
               'unitType': 'kWh'},
              {'classifier': 'LINEAR_INTERPOLATION',
               'datapointType': 'interval',
               'description': 'No descri

## create ewx datasource

In [3]:
domain.Datasource.from_message(ds_dict)

Property name description can not be mapped to a domain attribute on domain class EnergyworxDomain
Attribute None can not be mapped to a property instance on domain class EnergyworxDomain
[{'validFrom': '1900-01-01T00:00:00.000000', 'description': '', 'tagLinks': [''], 'version': '2019-12-03T19:16:17.000000', 'tag': 'premise', 'createdBy': 'System', 'removed': False, 'properties': [{'deleted': False, 'value': 'MARLBOROUGH', 'readOnly': False, 'value_type': 'string', 'key': 'city', 'type': 'string'}, {'deleted': False, 'value': 'TIERPOINT-MA', 'readOnly': False, 'value_type': 'string', 'key': 'name', 'type': 'string'}, {'deleted': False, 'value': '01752', 'readOnly': False, 'value_type': 'string', 'key': 'zip', 'type': 'string'}, {'deleted': False, 'value': 'USA', 'readOnly': False, 'value_type': 'string', 'key': 'country', 'type': 'string'}, {'deleted': False, 'value': 'Unknown', 'readOnly': False, 'value_type': 'string', 'key': 'county', 'type': 'string'}, {'deleted': False, 'value': 

{None: 'NEPOOL_MECO_0219222018'}

### datasource debug

In [18]:
dmc = base.EnergyworxDomainMeta('EnergyworxDomain', (base.EnergyworxDomain, ), ds_dict)

#new_class = super(dmc)
new_class = dmc
attr_by_prop_name = {}
prop_by_attr = {}
for attr, value in ds_dict.items():
    
    if type(value) != list:
        value_obj = base.Property(value)
        if isinstance(value_obj, base.Property):
            setattr(value_obj, 'attr_name', attr)
            attr_by_prop_name.update({value_obj.name: attr})
            prop_by_attr.update({attr: value_obj})
            
    elif type(value) == list:
        struct_prop = []
        if attr == 'tags':
            for v in value:
                struct_prop.append(domain.Tag.from_dict(v))
            struct_prop = base.StructuredProperty(struct_prop)
        
            if isinstance(struct_prop, base.StructuredProperty):
                setattr(struct_prop, 'attr_name', attr)
                attr_by_prop_name.update({'tags': attr})
                prop_by_attr.update({attr: struct_prop})
            
        elif attr == 'channels':

            for v in value:
                new_ch = domain.Channel()
                try:
                    new_ch.description = v['description']
                except:
                    new_ch.description = ''
                new_ch.is_source = v['isSource']
                new_ch.classifier = v['classifier']
                new_ch.unit_type = v['unitType']
                new_ch.datapoint_type = v['datapointType']
                new_ch.id = v['id']
                new_ch.name = v['name']
                struct_prop.append(new_ch)
            struct_prop = base.StructuredProperty(struct_prop)
            
            if isinstance(struct_prop, base.StructuredProperty):
                setattr(struct_prop, 'attr_name', attr)
                attr_by_prop_name.update({'channels': attr})
                prop_by_attr.update({attr: struct_prop})
        

if attr_by_prop_name:
    new_class.attr_by_prop_name = attr_by_prop_name

if prop_by_attr:
    new_class.prop_by_attr = prop_by_attr

In [21]:
attr_by_prop_name

{'': 'description',
 'tags': 'tags',
 'NEPOOL_MECO_0219222018': 'name',
 'channels': 'channels',
 False: 'readOnly',
 'America/New_York': 'timezone',
 'ACCOUNT': 'classifier'}

In [17]:
prop_by_attr

{'description': <energyworx_public.base.Property at 0x296f72bcec8>,
 'tags': <energyworx_public.base.StructuredProperty at 0x296f72e09c8>,
 'id': <energyworx_public.base.Property at 0x296f72eaa48>,
 'channels': <energyworx_public.base.StructuredProperty at 0x296f72eaac8>,
 'readOnly': <energyworx_public.base.Property at 0x296f72eaec8>,
 'timezone': <energyworx_public.base.Property at 0x296f72eaf08>,
 'classifier': <energyworx_public.base.Property at 0x296f72eaf48>,
 'name': <energyworx_public.base.Property at 0x296f72eaf88>}

In [12]:
dmc

energyworx_public.base.EnergyworxDomain

## datasource object

In [92]:
#tags
ds_tag_obj = []
for t in ds_dict['tags']:
    ds_tag_obj.append(domain.Tag.from_dict(t))

#channels
ds_ch_obj = []

for c in ds_dict['channels']:
        new_ch = domain.Channel()
        try:
            new_ch.description = c['description']
        except:
            new_ch.description = ''
        new_ch.is_source = c['isSource']
        new_ch.classifier = c['classifier']
        new_ch.unit_type = c['unitType']
        new_ch.datapoint_type = c['datapointType']
        new_ch.id = c['id']
        new_ch.name = c['name']
        ds_ch_obj.append(new_ch)

#datasource object
datasource = domain.Datasource()

datasource.channels = ds_ch_obj
datasource.tags = base.StructuredProperty(ds_tag_obj)
datasource.name = base.Property(ds_dict['name'])
datasource.id = base.Property(ds_dict['id'])
datasource.classifier = base.Property(ds_dict['classifier'])
datasource.description = base.Property(ds_dict['description'])
print('created datasource manually.')
datasource

created datasource manually.


In [89]:
print('channels are:')
print('-------------------')
for c in ds_ch_obj:
    print(c.id)

channels are:
-------------------
DELIVERY_IDR
DELIVERY_SCALAR
FORECAST
LINEAR_INTERPOLATION
LIKE_DAY_SUBSTITUTION
MERGED
DELIVERY_SCALAR_HEARTBEAT
TOTAL_DEMAND
TOTAL_DEMAND_HEARTBEAT
DELIVERY_IDR_HEARTBEAT
ICAP
ICAP_HEARTBEAT
NITS
NITS_HEARTBEAT
VEE_RESULT


In [60]:
datasource.__class__.__bases__

(energyworx_public.base.EnergyworxDomain,)